In [1]:
import kagglehub
kagglehub.login()

Kaggle credentials set.
Kaggle credentials successfully validated.


In [2]:
isic_2024_challenge_path = kagglehub.competition_download('isic-2024-challenge')

100%|██████████| 2.00G/2.00G [00:11<00:00, 186MB/s]

Extracting files...


In [8]:
##############
# LIBRARIES
##############

import pandas as pd
import numpy as np

from sklearn.preprocessing import RobustScaler

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings("ignore")

# Adjusting Row Column Settings

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.width', 500)

In [9]:
############
# LOAD DATA
############

train_data_path = "/content/drive/MyDrive/Skin Cancer Detection/exploratory_data_analysis.csv"
train_data = pd.read_csv(train_data_path)

train_image_dir = f"{isic_2024_challenge_path}/train-image"

In [10]:
#################
# MISSING VALUES
#################

train_data = train_data[~((train_data['target'] == 0) & (train_data.isnull().any(axis=1)))]

train_data['age_approx'] = train_data['age_approx'].fillna(train_data['age_approx'].median())
train_data['sex'] = train_data['sex'].fillna(train_data['sex'].mode()[0])

In [11]:
#####################
# ENCODING / SCALING
#####################

one_hot_cols = ['tbp_tile_type', 'anatom_site_general', 'sex']
train_data = pd.get_dummies(train_data, columns=one_hot_cols, drop_first=True, dtype=int)


columns_to_scale = train_data.select_dtypes(include=['float64', 'int64']).columns.difference(['target', 'images_path'])
scaler = RobustScaler()
train_data[columns_to_scale] = scaler.fit_transform(train_data[columns_to_scale])


# Rename
train_data.rename(columns={'tbp_tile_type_3D: white': 'tbp_tile_type_3D_white'}, inplace=True)

## Preprocessing

In [12]:
##############
# EXPORT DATA
##############

file_path = '/content/drive/MyDrive/Skin Cancer Detection/preprocessing_data.csv'
train_data.to_csv(file_path, index=False)